In [57]:
import pandas as pd
from datetime import datetime, timedelta
import matplotlib.pyplot as plt
from data_load import load_aggregate_data, get_user_tasks
from distance_map import load_distance_map
from gridload import load_grid_graph, plot_grid
from data_analyze import calculate_metrics
import numpy as np
import json

# EDA with Receiving -> PTC Data

In [80]:
df = pd.read_csv("Dock-PTC Data/DC820 to PTC.csv")

## Remove/rename columns

In [81]:
cols = [df.columns[0], "WHSE", "NBR_OF_CASES", "NBR_UNITS", "MENU_OPTN_NAME"]
for col in cols:
    try:
        df.drop(columns=[col], inplace=True)
    except:
        continue
df

,CNTR_NBR,TO_LOCN,FROM_LOCN,CREATE_DATE,USER_ID
0,BM9325112,631,075R,2024-02-19 00:10,ggonzale
1,BM9325114,631,075R,2024-02-19 00:10,ggonzale
2,BM9325111,631,075R,2024-02-19 00:11,ggonzale
3,BM9325110,631,075R,2024-02-19 00:15,ggonzale
4,BM9325108,631,075R,2024-02-19 00:22,ggonzale
...,...,...,...,...,...
91444,BM8383721,631,061R,2024-06-17 15:43,ebrea
91445,BH9618610,641,NaN,2024-06-17 15:43,achampion
91446,BM9219077,641,137R,2024-06-17 15:43,rpierre
91447,BM8384582,641,051R,2024-06-17 15:44,jkanicki


In [82]:
renames = {
    "CNTR_NBR" : "id", 
    "TO_LOCN" : "to_locn",
    "FROM_LOCN" : "from_locn",
    "CREATE_DATE" : "from_time",
    "USER_ID" : "user",
}
try:
    df.rename(columns=renames, inplace=True)
except:
    print("Rename not successful")

## Format times and clean bad rows

In [70]:
df["from_time"] = pd.to_datetime(df["from_time"])

In [83]:
original = df.shape[0]
df = df[df["to_locn"] != df["from_locn"]]
new = df.shape[0]
print(f"Removed {original - new} rows")

Removed 4302 rows


In [84]:
original = df.shape[0]
df.dropna(subset=["from_locn"], inplace=True)
new = df.shape[0]
print(f"Removed {original - new} rows")

Removed 2358 rows


C:\Users\ctang\AppData\Local\Temp\1\ipykernel_17056\137836925.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df.dropna(subset=["from_locn"], inplace=True)


## Exploring data in cols

In [85]:
df1 = df[df["from_locn"].str.contains("692")]
df1

,id,to_locn,from_locn,from_time,user
1909,BW9803259,631,692002,2024-02-21 17:13,eperez1
1910,BW9803258,631,692009,2024-02-21 17:13,eperez1
1911,BW9803257,631,692029,2024-02-21 17:13,eperez1
1912,BW9803256,631,692036,2024-02-21 17:13,eperez1
1913,BW9803255,631,692037,2024-02-21 17:13,eperez1
1914,BW9803254,631,692040,2024-02-21 17:14,eperez1
1915,BW9803253,631,692045,2024-02-21 17:14,eperez1
1916,BW9803252,631,692061,2024-02-21 17:14,eperez1
1945,BW9661946,631,692034,2024-02-21 17:34,eperez1
1946,BW9661945,631,692044,2024-02-21 17:35,eperez1


In [87]:
set(df["from_locn"])

{'002A003',
 '002A008',
 '002A014',
 '002A029',
 '002A030',
 '002A036',
 '002A037',
 '002A039',
 '002B017',
 '002R',
 '003A007',
 '003A008',
 '003A017',
 '003A024',
 '003A027',
 '003B001',
 '003B004',
 '003C015',
 '003D011',
 '003D016',
 '003E041',
 '003L001',
 '003L004',
 '003R',
 '004A025',
 '004A027',
 '004A030',
 '004A039',
 '004B036',
 '004E027',
 '004L001',
 '004L002',
 '004R',
 '005A002',
 '005A005',
 '005A006',
 '005A031',
 '005B000',
 '005B018',
 '005C025',
 '005L000',
 '005L004',
 '005M003',
 '005R',
 '006A003',
 '006A005',
 '006A021',
 '006A024',
 '006B021',
 '006B038',
 '006R',
 '007A009',
 '007A010',
 '007A017',
 '007A020',
 '007A021',
 '007A030',
 '007A033',
 '007A039',
 '007C006',
 '007D005',
 '007E014',
 '007M001',
 '007M004',
 '007R',
 '008A007',
 '008A030',
 '008A031',
 '008A038',
 '008L001',
 '008L002',
 '008R',
 '009A000',
 '009A001',
 '009A008',
 '009A010',
 '009A034',
 '009A040',
 '009A042',
 '009B020',
 '009B032',
 '009L003',
 '009R',
 '010A000',
 '010A015',
 '01

## Filter out specifics from location column and standardize values

In [88]:
# filter out from_locn
df = df[ (df['from_locn'].str.endswith('R')) |  # Condition 1
                        (df['from_locn'].str.startswith('C')) |  # Condition 2
                        (df['from_locn'].str.endswith('S')) |  # Condition 3
                        (df['from_locn'].isin(['BRK','INBP','MNOB', 'BRK1'])) |  # Condition 4
                        (df['from_locn'].astype(str).str.contains('692|682')) | 
                        (df['from_locn'].str.startswith('6')) | 
                        (df['from_locn'].isin([611, 621, 631, 641, 651, 661, 671, 681, 691]))]  
# remove trailing Rs and make 682/692*** onnly 682
def h(x):
    if x[-1] == 'R':
        return x[:-1]
    if x[-1] == 'S':
        return x[:-1]
    elif len(x) == 6 and x[0] == '6':
        return x[:3]
    else:
        return x
df["from_locn"] = df["from_locn"].apply(h)
df["from_locn"]

0        075
1        075
2        075
3        075
4        075
        ... 
91443    113
91444    061
91446    137
91447    051
91448    117
Name: from_locn, Length: 84154, dtype: object

In [92]:
set(df["to_locn"])

{'631', '641', '651', '661'}

In [91]:
# filter to_locn so only between 600 and 700
df = df[df["to_locn"].isin(["611", "621", "631", "641", "651", "661", "671", "681", "691"])]